In [18]:
import os
import time
import pandas as pd
import urllib.request
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException


In [19]:

ean_list = ['1056032', 'B07PBWCSC8', 'B006E9VVT0', 'B07JXYYZL2', 'B06XDMXQ98', 'M9697', '1056032', 'B07KYM9KF5']

os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'


In [20]:
def bullet_points(all_bullets): 
    
    bullet1 = bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 =  float('nan') 
    
    if len(all_bullets.split("\n")) == 8: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8= [str(e) for e in all_bullets.split("\n")]
    elif len(all_bullets.split("\n")) == 7: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7= [str(e) for e in all_bullets.split("\n")] 
    elif len(all_bullets.split("\n")) == 6: 
        bullet1, bullet2, bullet3, bullet4, bullet5, bullet6 = [str(e) for e in all_bullets.split("\n")] 
    elif len(all_bullets.split("\n")) == 5: 
        bullet1, bullet2, bullet3, bullet4, bullet5 = [str(e) for e in all_bullets.split("\n")] 
    elif len(all_bullets.split("\n")) == 4: 
        bullet1, bullet2, bullet3, bullet4 = [str(e) for e in all_bullets.split("\n")] 
    elif len(all_bullets.split("\n")) == 3: 
        bullet1, bullet2, bullet3, = [str(e) for e in all_bullets.split("\n")]
    elif len(all_bullets.split("\n")) == 2: 
        bullet1, bullet2 = [str(e) for e in all_bullets.split("\n")]
    elif len(all_bullets.split("\n")) == 1: 
        bullet1 = [str(e) for e in all_bullets.split("\n")]
    
    return bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8  


def asignar_variables_amazon(detalles_tecnicos, detalles_usuarios_amazon):
    title = price = bullet1 = bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 = main_image = marca = numero_de_modelo = valoracion = clasificacion = producto_desde = float('nan')
        
    for detalle_tecnico in detalles_tecnicos:
        if detalle_tecnico[0] == 'Marca':
            marca = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Número de modelo':
            numero_de_modelo = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Valoración media de los clientes':
            valoracion = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Clasificación en los más vendidos de Amazon':
            clasificacion = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Producto en Amazon.es desde':
            producto_desde = detalle_tecnico[1]
    
    for detalle_usuarios_amazon in detalles_usuarios_amazon:
        
        if detalle_usuarios_amazon[0] == 'Marca':
            marca = detalle_usuarios_amazon[1]
        elif detalle_usuarios_amazon[0] == 'Número de modelo':
            numero_de_modelo = detalle_usuarios_amazon[1]
        elif detalle_usuarios_amazon[0] == 'Valoración media de los clientes':
            valoracion = detalle_usuarios_amazon[1]
        elif detalle_usuarios_amazon[0] == 'Clasificación en los más vendidos de Amazon':
            clasificacion = detalle_usuarios_amazon[1]
        elif detalle_usuarios_amazon[0] == 'Producto en Amazon.es desde':
            producto_desde = detalle_usuarios_amazon[1]
    
    return marca, numero_de_modelo, valoracion, clasificacion, producto_desde


def initialize_amazon_info_df():
    
    amazon_info = pd.DataFrame(columns = ['item_name.value', 
                                                          'list_price.value_with_tax' , 'Bullet_point.value',
                                                          'Bullet_point#2.value','Bullet_point#3.value',
                                                          'Bullet_point#4.value', 'Bullet_point#5.value',
                                                          'Bullet_point#6.value','Bullet_point#7.value',
                                                          'main_image','Marca','Número de modelo',
                                                          'Valoración media de los clientes', 
                                                          'Clasificación en los más vendidos de Amazon',
                                                          'Producto en Amazon.es desde'
                                                         ])
    return amazon_info



In [25]:

def amazon(ean_list):
    
    amazon_info = initialize_amazon_info_df()
            
    browser = webdriver.Firefox()
    browser.get('https://www.amazon.es')
    
    time.sleep(3)
    
    for ean in ean_list:
        
        title = price = bullet1 = bullet2 = bullet3 = bullet4 = bullet5 = bullet6 = bullet7 = bullet8 = main_image = marca = numero_de_modelo = valoracion = clasificacion = producto_desde = float('nan')

        
        search = browser.find_element_by_xpath('//input[@id="twotabsearchtextbox"]')
        search.clear()
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)
        
        
        try:
    
            action = ActionChains(browser)
            click_image  = browser.find_element_by_xpath("(//div[@class='a-section aok-relative s-image-fixed-height'])")
            action.move_to_element((click_image)).click().perform()
        
        except NoSuchElementException:
            
            action = ActionChains(browser)
            click_image  = browser.find_element_by_xpath("(//div[@class='a-section aok-relative s-image-square-aspect'])")
            action.move_to_element((click_image)).click().perform()
            
        except NoSuchElementException:
            
            action = ActionChains(browser)
            click_image  = browser.find_element_by_xpath("(//div[@class='a-section aok-relative s-image-tall-aspect'])")
            action.move_to_element((click_image)).click().perform()

        except NoSuchElementException:
            
            print(f"Item {ean} not found")
        
        time.sleep(3)
        
        
        try: 
            
            title = browser.find_elements_by_xpath('//h1[@id="title"]')[0].text
        
        except NoSuchElementException:
            
            print(f"title {ean} not found")
            
        except IndexError:
            
            print(f"title {ean} not found")
            
            
        try: 
            
            price = browser.find_elements_by_xpath('//span[@id="priceblock_ourprice"]')[0].text
            
        except NoSuchElementException:
            
            price = browser.find_elements_by_xpath('//span[@id="priceblock_dealprice"]')[0].text
            
        except NoSuchElementException:
            
            price = browser.find_elements_by_xpath('//span[@id="priceblock_saleprice"]')[0].text
            
        except NoSuchElementException:
            
            print(f"price {ean} not found")
        
        except IndexError:
            
            print(f"price {ean} not found")
            
            
        
        try:
            
            all_bullets = browser.find_elements_by_xpath('//div[@id="feature-bullets"]')[0].text
            bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, bullet8 = bullet_points(all_bullets)
        
        except NoSuchElementException:
            
            print(f"bullets {ean} not found")
        
        except IndexError:
            
            print(f"bullets {ean} not found")
            
        
        try: 
            
            main_image = browser.find_element_by_xpath('//img[@id="landingImage"]').get_attribute('src')
    
        except NoSuchElementException:
            
            print(f"main image {ean} not found")
        
        except IndexError:
            
            print(f"main image {ean} not found")
            
            
        try:
            
            url = str(browser.current_url)
            time.sleep(3)
            lista = pd.read_html(url, match='.+', flavor = None, header = None, index_col =None, skiprows=None, attrs=None, parse_dates=False, thousands=', ', encoding=None, decimal='.', converters=None, na_values=None, keep_default_na=True, displayed_only=True)
            time.sleep(3)

            detalles_tecnicos = lista[1].values.tolist()   
            detalles_usuarios_amazon = lista[2].values.tolist()  

            marca, numero_de_modelo, valoracion, clasificacion, producto_desde = asignar_variables_amazon(detalles_tecnicos, detalles_usuarios_amazon)

                    
            
        except urllib.error.HTTPError:
            print(f"detalles tecnicos {ean} not found")
                
        amazon_info.loc[ean] = [title, price, bullet1, bullet2, bullet3, bullet4, bullet5, bullet6, bullet7, main_image, marca, numero_de_modelo, valoracion, clasificacion, producto_desde]
        
        
    browser.close()

    return amazon_info


In [24]:
df= amazon(ean_list)

price not found
bullets not found
main image not found


In [26]:
df

,item_name.value,list_price.value_with_tax,Bullet_point.value,Bullet_point#2.value,Bullet_point#3.value,Bullet_point#4.value,Bullet_point#5.value,Bullet_point#6.value,Bullet_point#7.value,main_image,Marca,Número de modelo,Valoración media de los clientes,Clasificación en los más vendidos de Amazon,Producto en Amazon.es desde
1056032,Memoria RAM de 4GB para Acer Aspire AS7250-040...,"34,22 €",RAM Memory,› Ver más detalles,NaN,NaN,NaN,NaN,NaN,https://images-na.ssl-images-amazon.com/images...,Offtek,NaN,Sea la primera persona en escribir una opinión,NaN,24 de agosto de 2012
B07PBWCSC8,"Profesional Blanqueamiento Dental Gel,Blanquea...","14,99 €",Eliminar Años de Manchas: Nuestro lápiz blanqu...,Conveniente & Amigable para Viajes: No se nece...,Seguro & Suave para Uso Diario: ¡Este lápiz pa...,Usas: 1. Use un cepillo de dientes o hilo dent...,Calidad Profesional & Asequible: Nuestro lápiz...,NaN,NaN,https://images-na.ssl-images-amazon.com/images...,NaN,NaN,NaN,NaN,NaN
B006E9VVT0,El Decameron Versión Kindle,"14,99 €",Eliminar Años de Manchas: Nuestro lápiz blanqu...,Conveniente & Amigable para Viajes: No se nece...,Seguro & Suave para Uso Diario: ¡Este lápiz pa...,Usas: 1. Use un cepillo de dientes o hilo dent...,Calidad Profesional & Asequible: Nuestro lápiz...,NaN,NaN,https://images-na.ssl-images-amazon.com/images...,NaN,NaN,NaN,NaN,NaN
B07JXYYZL2,heekpek Guantes de Deportes de Invierno para H...,"9,99 €","Material: PU, poliéster, terciopelo. Alta cali...","TAMAÑO: S (dedo medio: 2.95 ""/7.5cm, total: 8....",Forro de PU y lana,Diseño: Cremallera trasera y cremallera adicio...,Accesorio necesario para actividades al aire l...,Nuevas tendencias de moda para la temporada de...,NaN,https://images-na.ssl-images-amazon.com/images...,NaN,NaN,NaN,NaN,NaN
B06XDMXQ98,Shlax&Wing La Moda único Hombre Seda Corbatas ...,"6,99 €",Esta opción incluye una corbata de tamaño clás...,Tamaño de la corbata: 147cm x 8cm.,Seda,La corbata a cuadros es ideal para fines menos...,Nuestros productos están tejidos en jacquard e...,Fuimos fundados originalmente en 2008. Llevamo...,NaN,https://images-na.ssl-images-amazon.com/images...,NaN,NaN,NaN,NaN,NaN
M9697,"Converse Chuck Taylor All Star Core Ox, Zapati...","35,88 € - 317,48 €",Material exterior: Lona,Material de la suela: Goma,Cierre: Cordones,Tipo de tacón: Plano,NaN,NaN,NaN,https://images-na.ssl-images-amazon.com/images...,NaN,NaN,NaN,NaN,NaN
B07KYM9KF5,TECKIN Bombilla Inteligente LED WiFi con luz c...,"15,99 €","【Control remoto】Encluso si no está en casa, pu...",【Control de voz】Controle . Simplemente su bomb...,【Millones de colores cambiando】Exprésase y haz...,【Ahorro de energía y Amplio Uso】Con tecnología...,【Larga vida útil】Para uso normal (3 horas / dí...,NaN,NaN,https://images-na.ssl-images-amazon.com/images...,TECKIN,NaN,"4,2 de 5 estrellas428 valoraciones de clientes",nº176 en Iluminación (Ver el Top 100 en Ilumin...,15 de diciembre de 2018


In [ ]:
Hace el try except por cada elemento, para el tema de la imagen buscar por titulo, siempre centrarse en la clase, es lo que no cambian. 